In [84]:
import pandas as pd
import tensorflow as tf
import numpy as np
import sklearn

In [86]:
from tensorflow.keras.layers import Dense, Dropout, Activation, Input
from tensorflow.keras.models import Model
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [88]:
base = pd.read_csv("games.csv")

In [90]:
base

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
#queremos apenas os da america, dropando o que nao ira ser utilizada na rede neural
base = base.drop("Other_Sales", axis=1)
base = base.drop("Global_Sales", axis=1)
base = base.drop("Developer", axis=1)

In [94]:
base.shape

(16719, 13)

In [96]:
base.isnull().sum()

Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Rating             6769
dtype: int64

In [98]:
#excluir as linhas com valores nulos
base = base.dropna(axis=0)

In [100]:
base.shape #metade da base de daos foi apagada

(6825, 13)

In [102]:
base.isnull().sum() #nenhum valor nulo anteriormente exlcuidos

Name               0
Platform           0
Year_of_Release    0
Genre              0
Publisher          0
NA_Sales           0
EU_Sales           0
JP_Sales           0
Critic_Score       0
Critic_Count       0
User_Score         0
User_Count         0
Rating             0
dtype: int64

In [104]:
base["Name"].value_counts()

Name
Need for Speed: Most Wanted                  8
Madden NFL 07                                8
LEGO Star Wars II: The Original Trilogy      8
The Sims 2                                   7
Terraria                                     7
                                            ..
Castlevania: Portrait of Ruin                1
Suzuki TT Superbikes                         1
Rumble Roses                                 1
Sherlock Holmes: The Mystery of the Mummy    1
STORM: Frontline Nation                      1
Name: count, Length: 4377, dtype: int64

In [106]:
base = base.drop("Name", axis=1)

In [108]:
base.shape

(6825, 12)

In [110]:
x = base.iloc[:,[0,1,2,3,7,8,9,10,11]].values
x

array([['Wii', 2006.0, 'Sports', ..., '8', 322.0, 'E'],
       ['Wii', 2008.0, 'Racing', ..., '8.3', 709.0, 'E'],
       ['Wii', 2009.0, 'Sports', ..., '8', 192.0, 'E'],
       ...,
       ['PC', 2014.0, 'Action', ..., '7.6', 412.0, 'M'],
       ['PC', 2011.0, 'Shooter', ..., '5.8', 43.0, 'T'],
       ['PC', 2011.0, 'Strategy', ..., '7.2', 13.0, 'E10+']], dtype=object)

In [112]:
base.columns

Index(['Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Critic_Score', 'Critic_Count', 'User_Score',
       'User_Count', 'Rating'],
      dtype='object')

In [114]:
y_na = base.iloc[:,4].values
y_eu = base.iloc[:,5].values
y_jp = base.iloc[:,6].values

In [116]:
y_na

array([4.136e+01, 1.568e+01, 1.561e+01, ..., 0.000e+00, 1.000e-02,
       0.000e+00])

In [118]:
y_eu

array([2.896e+01, 1.276e+01, 1.093e+01, ..., 1.000e-02, 0.000e+00,
       1.000e-02])

In [120]:
y_jp

array([3.77, 3.79, 3.28, ..., 0.  , 0.  , 0.  ])

In [122]:
base["Platform"].value_counts #ps2 1 0 0 0, x360 0 1 0 0 ...

<bound method IndexOpsMixin.value_counts of 0        Wii
2        Wii
3        Wii
6         DS
7        Wii
        ... 
16667    GBA
16677    GBA
16696     PC
16700     PC
16706     PC
Name: Platform, Length: 6825, dtype: object>

In [124]:
onehotencoder = ColumnTransformer(transformers=[("OneHot",OneHotEncoder(), [0,2,3,8])],remainder="passthrough")

In [126]:
x = onehotencoder.fit_transform(x).toarray()

In [128]:
x.shape

(6825, 303)

In [130]:
x[0] #atributos que ja existiam mais as aplicacoes do one hot encoder

array([0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 

In [134]:
#diferentemten dos processor anteriores, aqui estamos lidando com mais valores, assim sendo preferivel este modelo
camada_entrada = Input(shape=(303,))
camada_oculta1 = Dense(units=153,activation="relu")(camada_entrada) #303+3 / 2 = 153
camada_oculta2 = Dense(units=153,activation="relu")(camada_oculta1)
camada_saida1 = Dense(units=1,activation = "linear")(camada_oculta2)
camada_saida2 = Dense(units=1,activation = "linear")(camada_oculta2)
camada_saida3 = Dense(units=1,activation = "linear")(camada_oculta2)

In [136]:
regressor = Model(inputs=camada_entrada,outputs=[camada_saida1,camada_saida2,camada_saida3])

In [138]:
#problema mais complexo, utilizando o mean squared erros(mse)
regressor.compile(optimizer = "Adam", loss="mse")

In [140]:
regressor.fit(x,[y_na,y_eu,y_jp], epochs=500, batch_size=100)

Epoch 1/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - dense_7_loss: 2435.6938 - dense_8_loss: 652.9106 - dense_9_loss: 2395.2415 - loss: 5484.4043      
Epoch 2/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - dense_7_loss: 3.4954 - dense_8_loss: 3.3010 - dense_9_loss: 2.9328 - loss: 9.7306 
Epoch 3/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_7_loss: 3.2407 - dense_8_loss: 2.9177 - dense_9_loss: 1.5879 - loss: 7.7476
Epoch 4/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - dense_7_loss: 7.1752 - dense_8_loss: 4.4116 - dense_9_loss: 8.2854 - loss: 19.8752
Epoch 5/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - dense_7_loss: 5.2827 - dense_8_loss: 1.8153 - dense_9_loss: 8.4046 - loss: 15.5060 
Epoch 6/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - dense_7_loss: 2.7617 - dense_8_loss: 1.2055 - dense_9_loss: 1.3861 - loss: 5.3538
Epoch 7/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - dense_7_loss: 1.4895 - dense_8_loss: 1.3076 - dense_9_loss: 0.3883 - loss: 3.1863
Epoch 8/500
69/69 ━━━━━━━━━━━━━━━━

In [142]:
#Todas as linhas abaixo sao comparativos do proprio sistema com a rede neural criada no codigo
previsao_na, previsao_eu,previso_jp = regressor.predict(x)

214/214 ━━━━━━━━━━━━━━━━━━━━ 0s 368us/step


In [153]:
previsao_na, previsao_na.mean()

(array([[ 1.2401998e+01],
        [ 1.2915152e+01],
        [ 1.0770203e+01],
        ...,
        [ 1.7019030e-01],
        [-9.2608929e-03],
        [-1.6432881e-01]], dtype=float32),
 0.45105824)

In [169]:
y_na, y_na.mean()

(array([4.136e+01, 1.568e+01, 1.561e+01, ..., 0.000e+00, 1.000e-02,
        0.000e+00]),
 0.3944835164835165)

In [175]:
mean_absolute_error(y_na, previsao_na)

0.2910475109787651

In [155]:
previsao_eu, previsao_eu.mean()

(array([[ 9.2506962e+00],
        [ 9.4412823e+00],
        [ 7.7364783e+00],
        ...,
        [ 2.2323781e-01],
        [ 7.8172386e-03],
        [-8.9277118e-02]], dtype=float32),
 0.27581805)

In [171]:
y_eu, y_eu.mean()

(array([2.896e+01, 1.276e+01, 1.093e+01, ..., 1.000e-02, 0.000e+00,
        1.000e-02]),
 0.23608937728937732)

In [177]:
mean_absolute_error(y_eu, previsao_eu)

0.20697684982495426

In [179]:
previso_jp, previso_jp.mean()

(array([[ 4.134156  ],
        [ 4.2518663 ],
        [ 3.5311573 ],
        ...,
        [ 0.13840422],
        [ 0.01267457],
        [-0.01762415]], dtype=float32),
 0.08059338)

In [173]:
y_jp, y_jp.mean()

(array([3.77, 3.79, 3.28, ..., 0.  , 0.  , 0.  ]), 0.06415824175824175)

In [183]:
mean_absolute_error(y_jp, previso_jp)

0.09533370709720548

In [163]:
from sklearn.metrics import mean_absolute_error